# 3D Droplet Oscillation

Results published: hopefully at some point!

It is part of the BoSSS-long-term validation test suite, which consists of 
several computationally expensive test-cases (runtime in the order of days),
which are performed on a regular basis in order to validate the 
physical correctness of BoSSS simulations.


### Preliminaries

This example can be found in the source code repository as as `Droplet3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Timestepping;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ],,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,[ \\hpccluster\hpccluster-scratch\smuda\ == ],FDY\smuda,DC2,<null>,Normal,True


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [ ]:
//var myBatch = ExecutionQueues[0];
var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ]


A switch for running the worksheet locally (since we have a folder structure for the data, which is not copied on Jenkins for testing)

In [ ]:
bool NunitTest = true;

In [ ]:
//foreach(var s in wmg.Sessions)
//    s.Delete(true);

## Verification of Initial Value data

Initial values and parameters for the simulation (intial droplet shape, Ohnesorg number, initial velocity)
were specified by project partner (TU Graz, Group Prof. Brenn).
Details can be found in Document `setup.pdf`, to be found in the same directory as this worksheet.

First, it is verified that the initial values chosen here actually match the specification.

In [ ]:
//MultidimensionalArray[] ReferenceData = new MultidimensionalArray[5];
int[] modes = new int[] { 2, 3, 4 };
List<string> cases = new List<string>();
int nCase = 0;
// add simulations for mode 2
if(modes.Contains(2)) {
    cases.Add("m2_Oh01_eta07");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m2_Oh01_eta06");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m2_Oh01_eta05");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m2_Oh01_eta03");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
// add simulations for mode 3
if(modes.Contains(3)) {
    cases.Add("m3_Oh01_eta07");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m3_Oh01_eta06");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m3_Oh01_eta05");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m3_Oh01_eta03");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
// add simulations for mode 4
if(modes.Contains(4)) {
    cases.Add("m4_Oh01_eta07");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh01_eta06");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh01_eta05");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh01_eta03");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh01_eta02");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
int numCases = cases.Count;
numCases

case 1: m2_Oh01_eta07
case 2: m2_Oh01_eta06
case 3: m2_Oh01_eta05
case 4: m2_Oh01_eta03
case 5: m3_Oh01_eta07
case 6: m3_Oh01_eta06
case 7: m3_Oh01_eta05
case 8: m3_Oh01_eta03
case 9: m4_Oh01_eta07
case 10: m4_Oh01_eta06
case 11: m4_Oh01_eta05
case 12: m4_Oh01_eta03
case 13: m4_Oh01_eta02


13

In [ ]:
bool[] refDataProvided = new bool[numCases];
refDataProvided[3] = true;
//refDataProvided[7] = true;
refDataProvided[11] = true;
refDataProvided[12] = true;

Load reference data for all cases; These files contain two columns, i.e. the azimuth angle and the respective droplet radius.

In [ ]:
MultidimensionalArray[] ReferenceData = new MultidimensionalArray[numCases];
for(int iCase = 0; iCase < numCases; iCase++) {
    if(refDataProvided[iCase]) {
    if(!NunitTest)  
        ReferenceData[iCase] = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/surfaceDrop_{cases[iCase]}.txt");
    else
        ReferenceData[iCase] = IMatrixExtensions.LoadFromTextFile($"surfaceDrop_{cases[iCase]}.txt");
}
}

Analytical expressions for the reference data (see `setup.pdf`); this is to verify that the definition of Legendre
Functions resp. Polynomials in BoSSS actually matches the definition used by the TU Graz group.

In [ ]:
Dictionary<string, Func<double, double>> casesRadius = new Dictionary<string, Func<double, double>>();

In [ ]:
double caseR_m2eta07(double angle) {
   double radius = 0.895131 + 0.7*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta07", caseR_m2eta07);

In [ ]:
double caseR_m2eta06(double angle) {
   double radius = 0.923706 + 0.6*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta06", caseR_m2eta06);

In [ ]:
double caseR_m2eta05(double angle) {
   double radius = 0.947538 + 0.5*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta05", caseR_m2eta05);

In [ ]:
double caseR_m2eta03(double angle) {
    double radius = 0.981486 + 0.3*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m2_Oh01_eta03", caseR_m2eta03);

In [ ]:
double caseR_m3eta07(double angle) {
    double radius = 0.930122 + 0.7*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta07", caseR_m3eta07);

In [ ]:
double caseR_m3eta06(double angle) {
    double radius = 0.948619 + 0.6*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta06", caseR_m3eta06);

In [ ]:
double caseR_m3eta05(double angle) {
    double radius = 0.964301 + 0.5*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta05", caseR_m3eta05);

In [ ]:
double caseR_m3eta03(double angle) {
    double radius = 0.987144 + 0.3*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta03", caseR_m3eta03);

In [ ]:
double caseR_m4eta07(double angle) {
    double radius = 0.943440 + 0.7*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta07", caseR_m4eta07);

In [ ]:
double caseR_m4eta06(double angle) {
    double radius = 0.958674 + 0.6*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta06", caseR_m4eta06);

In [ ]:
double caseR_m4eta05(double angle) {
    double radius = 0.971459 + 0.5*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta05", caseR_m4eta05);

In [ ]:
double caseR_m4eta03(double angle) {
    double radius = 0.989838 + 0.3*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta03", caseR_m4eta03);

In [ ]:
double caseR_m4eta02(double angle) {
    double radius = 0.995508 + 0.2*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta02", caseR_m4eta02);

Conversion to cartesian coordinates in order to match the data and verification against analytical expression:

In [ ]:
double[][] refX = new double[numCases][];
double[][] refZ = new double[numCases][];
double[][] caseX = new double[numCases][];
double[][] caseZ = new double[numCases][];
for(int iCase = 0; iCase < numCases; iCase++) {

    if(refDataProvided[iCase]) {
    double[] angle = ReferenceData[iCase].GetColumn(0);
    double[] radius = ReferenceData[iCase].GetColumn(1);
       
    double RadiusErrorNorm = 0.0;
    int I = angle.Length;
    double[] x1 = new double[I], z1 = new double[I];
    double[] cx1 = new double[I], cz1 = new double[I];
    for(int i = 0; i < I; i++) {
        x1[i] = Math.Sin(angle[i])*radius[i];
        z1[i] = Math.Cos(angle[i])*radius[i];
        
        double radius_expr = casesRadius[cases[iCase]](angle[i]);
        RadiusErrorNorm += (radius[i] - radius_expr).Pow2();     
        
        cx1[i] = Math.Sin(angle[i])*radius_expr;
        cz1[i] = Math.Cos(angle[i])*radius_expr;
    } 
    refX[iCase] = x1;
    refZ[iCase] = z1;
    caseX[iCase] = cx1;
    caseZ[iCase] = cz1;

    RadiusErrorNorm = RadiusErrorNorm.Sqrt();
    Console.WriteLine($"Comparison error for radius in case {iCase + 1}: {RadiusErrorNorm}");
    // Note: since the factors in `setup.pdf` are only provided up to 6 digits, an error threshold of 1e-5 seems reasonable.
    Assert.LessOrEqual(RadiusErrorNorm, 1e-5, "Error in comparing reference data against Legendre polynomials in BoSSS");
}
}

Comparison error for radius in case 4: 5.062870042266827E-06
Comparison error for radius in case 12: 2.8677785870933128E-06
Comparison error for radius in case 13: 7.0258634464131154E-06


### Plot of Reference Data

In [ ]:
// int refCase = 4;
// Plot(refX[refCase], refZ[refCase], "Ref-Case", ". black",
//     caseX[refCase], caseZ[refCase], "Case", ". blue")

### Matching of the Spherical Harmonics against the provided Data

In [ ]:
Dictionary<string, Formula> casesPhi = new Dictionary<string, Formula>();

In [ ]:
var Phi_m2eta07_Init = new Formula(
"Phi1",
false,
"using ilPSP.Utils; " + 
"double Phi1(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.895131*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.7*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"}");
casesPhi.Add("m2_Oh01_eta07", Phi_m2eta07_Init);

In [ ]:
var Phi_m2eta06_Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.923706*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.6*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m2_Oh01_eta06", Phi_m2eta06_Init);

In [ ]:
var Phi_m2eta05_Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.947538*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.5*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m2_Oh01_eta05", Phi_m2eta05_Init);

In [ ]:
var Phi_m2eta03_Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.981486*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.3*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m2_Oh01_eta03", Phi_m2eta03_Init);

In [ ]:
var Phi_m3eta07_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.930122*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.7*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta07", Phi_m3eta07_Init);

In [ ]:
var Phi_m3eta06_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.948619*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.6*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta06", Phi_m3eta06_Init);

In [ ]:
var Phi_m3eta05_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.964301*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.5*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta05", Phi_m3eta05_Init);

In [ ]:
var Phi_m3eta03_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.987144*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.3*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta03", Phi_m3eta03_Init);

In [ ]:
var Phi_m4eta07_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.943440*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.7*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta07", Phi_m4eta07_Init);

In [ ]:
var Phi_m4eta06_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.958674*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.6*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta06", Phi_m4eta06_Init);

In [ ]:
var Phi_m4eta05_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.971459*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.5*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta05", Phi_m4eta05_Init);

In [ ]:
var Phi_m4eta03_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.989838*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.3*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta03", Phi_m4eta03_Init);

In [ ]:
var Phi_m4eta02_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.995508*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.2*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta02", Phi_m4eta02_Init);

In [ ]:
// IBoundaryAndInitialData[] Phi_iCase = new IBoundaryAndInitialData[]  { Phi1Init, Phi2Init, Phi3Init, Phi4Init, Phi5Init};

In [ ]:
for(int iCase = 0; iCase < numCases; iCase++) {
    if(refDataProvided[iCase]) {
    double[] angle = ReferenceData[iCase].GetColumn(0);
    //double[] xI = refX[iCase];   
    //double[] zI = refZ[iCase];
    int I = angle.Length;
    
    double PhiErr = 0;
    for(int i = 0; i < I; i++) {
        double radius_expr = casesRadius[cases[iCase]](angle[i]);    
        double x1 = Math.Sin(angle[i])*radius_expr;
        double z1 = Math.Cos(angle[i])*radius_expr;
    
        PhiErr += casesPhi[cases[iCase]].Evaluate(new Vector(x1, 0, z1), 0.0).Abs();
    }
    Console.WriteLine($"Phi error for case {iCase}: {PhiErr}");
    Assert.LessOrEqual(PhiErr, 1e-10, "Level-Set function is not zero at desired surface.");
    
    Assert.IsTrue(casesPhi[cases[iCase]].Evaluate(new Vector(1e-5, 1e-5, 1e-5), 0.0) < 0, "Inside must be phase A/negative");
    Assert.IsTrue(casesPhi[cases[iCase]].Evaluate(new Vector(1e+1, 1e+1, 1e+1), 0.0) > 0, "Outside must be phase B/positive");
}
}

Phi error for case 3: 1.7430501486614958E-14
Phi error for case 11: 2.6978419498391304E-14
Phi error for case 12: 1.965094753586527E-14


### Initial Velocities

In [ ]:
// //var anaVel = new IBoundaryAndInitialData[numCases,3];
// Dictionary<string, (IBoundaryAndInitialData velX, IBoundaryAndInitialData velY, IBoundaryAndInitialData velZ)> analyticVel = new Dictionary<string, (IBoundaryAndInitialData velX, IBoundaryAndInitialData velY, IBoundaryAndInitialData velZ)>();

// for(int iCase = 0; iCase < numCases; iCase++) {
//     MultidimensionalArray polVel;
//     MultidimensionalArray radVel;
//     if (!NunitTest) {
//         polVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/polarVel_{cases[iCase]}.txt");
//         radVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/radialVel_{cases[iCase]}.txt");
//     } else {
//         polVel = IMatrixExtensions.LoadFromTextFile($"polarVel_{cases[iCase]}.txt");
//         radVel = IMatrixExtensions.LoadFromTextFile($"radialVel_{cases[iCase]}.txt");
//     }
//     Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(0), radVel.GetColumn(0)));
//     Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(1), radVel.GetColumn(1)));
    
//     double[] radiusS = polVel.GetColumn(0);
//     double[] anglesS = polVel.GetColumn(1);
//     double[] polVelS = polVel.GetColumn(2);
//     double[] radVelS = radVel.GetColumn(2);
    
//     var velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
//     velX.SetData(anglesS, radiusS, polVelS, radVelS);
//     var velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
//     velY.SetData(anglesS, radiusS, polVelS, radVelS);
//     var velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
//     velZ.SetData(anglesS, radiusS, polVelS, radVelS);
    
//     // anaVel[iCase, 0] = velX;
//     // anaVel[iCase, 1] = velY;
//     // anaVel[iCase, 2] = velZ;
//     analyticVel.Add(cases[iCase], (velX, velY, velZ));
// }

## Grid Creation

In [ ]:
//foreach(var g in BoSSSshell.WorkflowMgm.Grids)
//   g.Delete(true);

### Quater-Domain grids
(Symmetry planes at $x = 0$ and $y = 0$)

In [ ]:
Dictionary<string, IGridInfo[]> gridTypes = new Dictionary<string, IGridInfo[]>();

In [ ]:
int[] Resolutions = new int[] { 6 };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];
double scale = 1.0;
for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];
    string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_wallBC_quarterDomain";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    //cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] xNodes = GenericBlas.Linspace(0, 3*scale, Res + 1);
        double[] yNodes = xNodes;
        double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, Res*2 + 1);
        
        var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        grd.Name = GridName;
        
        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;
            if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
                ret = IncompressibleBcType.SlipSymmetry.ToString();
            else
                ret = IncompressibleBcType.Wall.ToString();
            return ret;
        });        
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}
gridTypes.Add("wallBC", Grids);

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D'.
Grid Edge Tags changed.
An equivalent grid (3c07eba6-bc1c-4b2d-aea4-dfc90dc3f541) is already present in the database -- the grid will not be saved.


In [ ]:
gridTypes

key,value
wallBC,[ { Guid = 3c07eba6-bc1c-4b2d-aea4-dfc90dc3f541; Name = OscillatingDroplet3D_6x6x12_quarterDomain; Cell Count = 432; Dim = 3 } ]


In [ ]:
//var g = (wmg.Grids[0] as GridProxy).RealGrid;

In [ ]:
//(g.iGridData as GridData).GlobalBoundingBox

In [ ]:
//wmg.Sessions[0].Delete(true);

## Setup of control objects for all solver runs

In [ ]:
cases

index,value
0,m2_Oh01_eta07
1,m2_Oh01_eta06
2,m2_Oh01_eta05
3,m2_Oh01_eta03
4,m3_Oh01_eta07
5,m3_Oh01_eta06
6,m3_Oh01_eta05
7,m3_Oh01_eta03
8,m4_Oh01_eta07
9,m4_Oh01_eta06


In [ ]:
Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps)> casesSetUp = new Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps)>();

In [ ]:
casesSetUp.Add("m2_Oh01_eta07", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m2_Oh01_eta06", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m2_Oh01_eta05", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m2_Oh01_eta03", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m3_Oh01_eta07", (0.1, 2, 2e-3, 750));
casesSetUp.Add("m3_Oh01_eta06", (0.1, 2, 2e-3, 750));
casesSetUp.Add("m3_Oh01_eta05", (0.1, 2, 2e-3, 750));
casesSetUp.Add("m3_Oh01_eta03", (0.1, 2, 2e-3, 750));
casesSetUp.Add("m4_Oh01_eta07", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta06", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta05", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta03", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta02", (0.1, 1, 5e-3, 300));

In [ ]:
string[] gridTypeKeys = new string[] { "wallBC" }; ///{ "wallBC", "pressureOutletBC", "refinedTest" };

In [ ]:
List<XNSE_Control> Controls = new List<XNSE_Control>();
Controls.Clear();
int[] DegreeS = new int[] { 3 };
bool[] useInitial = new bool[] { false };
bool[] useNewton = new bool[] { false };

// string grdKey = gridTypeKeys[1];
// Grids = gridTypes[grdKey];

foreach(bool bInitial in useInitial) {
foreach(bool bNewton in useNewton) {
foreach(int k in DegreeS) {
foreach(string grdKey in gridTypeKeys) {
foreach(var grd in gridTypes[grdKey]) {
int iCase = 0;
foreach(var myCase in cases) {
    long J = grd.NumberOfCells;
    int AMRlvl = casesSetUp[myCase].AMRlevel;
    string JobName = $"OD3D_J{J}k{k}_{grdKey}_amr{AMRlvl}_{myCase}";
    if(bInitial) {
        JobName = JobName + "_thirdOrderInit";
    }
    if(bNewton) {
        JobName = JobName + "_Newton";
    }
    Console.WriteLine($"Case {iCase+1}: " + JobName);
    iCase++;

    var C = new XNSE_Control();
    
    C.SetGrid(grd);
    C.SetDGdegree(k);
    C.SessionName = JobName;
    
    C.InitialValues.Add("Phi", casesPhi[myCase]);
    
    C.PhysicalParameters.IncludeConvection = true;
    C.PhysicalParameters.rho_A = 1;
    C.PhysicalParameters.rho_B = 0.001;
    C.PhysicalParameters.mu_A = casesSetUp[myCase].Ohnesorge;
    C.PhysicalParameters.mu_B = casesSetUp[myCase].Ohnesorge/1000;
    C.PhysicalParameters.reynolds_B = 0.0;
    C.PhysicalParameters.reynolds_A = 0.0;
    C.PhysicalParameters.Sigma = 1;
    C.PhysicalParameters.pFree = 0.0;
    C.PhysicalParameters.mu_I = 0.0;
    C.PhysicalParameters.lambda_I = 0.0;
    C.PhysicalParameters.lambdaI_tilde = -1.0;
    C.PhysicalParameters.betaS_A = 0.0;
    C.PhysicalParameters.betaS_B = 0.0;
    C.PhysicalParameters.betaL = 0.0;
    C.PhysicalParameters.theta_e = 1.5707963267948966;
    C.PhysicalParameters.sliplength = 0.0;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.useArtificialSurfaceForce = false;
    
    C.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.StokesExtension;
    C.AdvancedDiscretizationOptions.SST_isotropicMode = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
    C.LSContiProjectionMethod = ContinuityProjectionOption.ConstrainedDG;
    
    C.TimeSteppingScheme = TimeSteppingScheme.BDF3;
    if(bNewton) {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    } else {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Picard;
    }
    C.NonLinearSolver.ConvergenceCriterion = 1e-9;
    C.NonLinearSolver.MinSolverIterations = 3;
    C.Timestepper_BDFinit = TimeStepperInit.SingleInit;
    C.Timestepper_LevelSetHandling = LevelSetHandling.Coupled_Once;
    C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
    C.dtFixed = casesSetUp[myCase].dt;
    C.NoOfTimesteps = casesSetUp[myCase].timesteps;
    
    if(AMRlvl > 0) {
        C.AdaptiveMeshRefinement = true;
        C.activeAMRlevelIndicators.Add(
            new AMRonNarrowband() { maxRefinementLevel = AMRlvl }
        );
    }
    
    // if(bInitial) {
    //     C.AddInitialValue("VelocityX#A", analyticVel[myCase].velX);
    //     //C.AddInitialValue("VelocityX#B", anaVel[myCase.Case - 1, 0]);
    //     C.AddInitialValue("VelocityY#A", analyticVel[myCase].velY);
    //     //C.AddInitialValue("VelocityY#B", anaVel[myCase.Case - 1, 1]);
    //     C.AddInitialValue("VelocityZ#A", analyticVel[myCase].velZ);
    //     //C.AddInitialValue("VelocityZ#B", anaVel[myCase.Case - 1, 2]);
    // }
    
    C.PostprocessingModules.Add(new SphericalHarmonicsLogging() { MaxL = 8, RotSymmetric = true });
    C.PostprocessingModules.Add(new DropletMetricsLogging() { AxisSymmetric = true });
    C.PostprocessingModules.Add(new EnergyLogging());
    
    C.TracingNamespaces = "*";
    
    Controls.Add(C);
    
}
}
}
}
}
}

Case 1: OD3D_J432k3_wallBC_amr1_m2_Oh01_eta07
Case 2: OD3D_J432k3_wallBC_amr1_m2_Oh01_eta06
Case 3: OD3D_J432k3_wallBC_amr1_m2_Oh01_eta05
Case 4: OD3D_J432k3_wallBC_amr1_m2_Oh01_eta03
Case 5: OD3D_J432k3_wallBC_amr2_m3_Oh01_eta07
Case 6: OD3D_J432k3_wallBC_amr2_m3_Oh01_eta06
Case 7: OD3D_J432k3_wallBC_amr2_m3_Oh01_eta05
Case 8: OD3D_J432k3_wallBC_amr2_m3_Oh01_eta03
Case 9: OD3D_J432k3_wallBC_amr1_m4_Oh01_eta07
Case 10: OD3D_J432k3_wallBC_amr1_m4_Oh01_eta06
Case 11: OD3D_J432k3_wallBC_amr1_m4_Oh01_eta05
Case 12: OD3D_J432k3_wallBC_amr1_m4_Oh01_eta03
Case 13: OD3D_J432k3_wallBC_amr1_m4_Oh01_eta02


In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal for " + i);
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal for " + i + " and " + j);
    }
}

## Launch Jobs

In [ ]:
Controls.Select(C => C.SessionName)

index,value
0,OD3D_J432k3_wallBC_amr1_m2_Oh01_eta07
1,OD3D_J432k3_wallBC_amr1_m2_Oh01_eta06
2,OD3D_J432k3_wallBC_amr1_m2_Oh01_eta05
3,OD3D_J432k3_wallBC_amr1_m2_Oh01_eta03
4,OD3D_J432k3_wallBC_amr2_m3_Oh01_eta07
5,OD3D_J432k3_wallBC_amr2_m3_Oh01_eta06
6,OD3D_J432k3_wallBC_amr2_m3_Oh01_eta05
7,OD3D_J432k3_wallBC_amr2_m3_Oh01_eta03
8,OD3D_J432k3_wallBC_amr1_m4_Oh01_eta07
9,OD3D_J432k3_wallBC_amr1_m4_Oh01_eta06


In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

In [ ]:
//wmg.AllJobs

In [ ]:
// wait for all jobs to finish (up to 5 days, check every 30 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600), PollingIntervallSeconds:(60*30));

In [ ]:
// detect failed Jobs in the job management
/*
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects
*/

In [ ]:
//suspects.Count()

In [ ]:
//NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
//BoSSSshell.WorkflowMgm.AllJobs.First().Value.ShowOutput();

In [ ]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()